In [1]:
import numpy as np
from skimage.transform import resize
from keras.applications.xception import Xception, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, RepeatVector, TimeDistributed
import pylidc as pl

Using TensorFlow backend.


In [2]:
feature_extractor = Xception(include_top=False, pooling='max')

In [3]:
num_slices = 50

In [4]:
model = Sequential()
model.add(CuDNNLSTM(100, input_shape=(num_slices, 2048)))
model.add(RepeatVector(num_slices))
model.add(CuDNNLSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
scans = pl.query(pl.Scan)
n = scans.count()

In [6]:
def get_annotated_slices(ann):
    zs = ann.bbox()[-1]
    return zs.start, zs.stop

In [22]:
def get_data(scans, i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.rollaxis(np.array([np.tile(resize(im.pixel_array, (331, 331), mode='constant'),
                                      (3, 1, 1)) for im in images]), 1, 4)
    nodule_zs = set.union(*(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))
    y = np.array([[i in nodule_zs for i in range(n)]])
    start = int((n - num_slices) / 2)
    X = X[start:start + num_slices]
    y = y[:, start:start + num_slices]
    y = np.expand_dims(y, axis=-1)
    return feature_extractor.predict(preprocess_input(X))[None], y

In [21]:
for _ in range(4):
    i = np.random.randint(n)
    X, y = get_data(i)
    model.fit(X, y, epochs=1)

Epoch 1/1
1/1 [==============================] - 0s 36ms/step - loss: 0.3839 - acc: 0.8400
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 0.1484 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 0.0874 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 0.0544 - acc: 1.0000


In [23]:
def datagen():
    scans = pl.query(pl.Scan)
    n = scans.count()
    i = np.random.randint(n)
    yield get_data(i)

In [24]:
model.fit_generator(datagen(), steps_per_epoch=1, epochs=2)

Epoch 1/2


ProgrammingError: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread.The object was created in thread id 4424 and this is thread id 11724 [SQL: 'SELECT count(*) AS count_1 \nFROM (SELECT scans.id AS scans_id, scans.study_instance_uid AS scans_study_instance_uid, scans.series_instance_uid AS scans_series_instance_uid, scans.patient_id AS scans_patient_id, scans.slice_thickness AS scans_slice_thickness, scans.pixel_spacing AS scans_pixel_spacing, scans.contrast_used AS scans_contrast_used, scans.is_from_initial AS scans_is_from_initial, scans.sorted_dicom_file_names AS scans_sorted_dicom_file_names \nFROM scans) AS anon_1'] [parameters: [{}]] (Background on this error at: http://sqlalche.me/e/f405)

In [18]:
X, y = get_data(np.random.randint(n))

In [19]:
model.fit(X, y, epochs=1)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6212 - acc: 1.0000


In [20]:
type(datagen())

generator